In [1]:
from ase.io.cfg import write_cfg
from ase.io.extxyz import read_xyz
import numpy as np


In [124]:
file = '/home/joe/mlip_test/gp_iter6_liqamo_lp.xyz'
db = list(read_xyz(file, index=slice(0,None)))
my = list(read_xyz('/home/joe/Documents/Ge_od/Potentials/Si_myDB_216/train_64_125_216_full.xyz', index=slice(0,None)))
Ge = list(read_xyz('/home/joe/Documents/Ge_od/Potentials/hp_liquid/hpliq_amoliq/hpliq_amoliq_train.xyz', index=slice(0,None)))

In [114]:
def write_cfg_db(f, a, force_name='dft_force', virial_name='dft_virial', energy_name='dft_energy'):
    """Writes a list of atoms to the MTP mlip-2 cfg database file format
    WARNING: incurs loss of array information other than positions and forces of atoms.
    Atoms.info dict is stored as features
    Params: set the key words for the forces, energy, and virials if they differ from the defaults above
    """
    f = open(f, 'w')

    for ct, val in enumerate(a):
        f.write('BEGIN_CFG\n Size\n')
        f.write('%i\n' % len(val))
        f.write('Supercell\n')
        cell = val.get_cell(complete=True)
        for i in range(3):
            f.write(('{:>10.6f}'*3 + '\n').format(*cell[i]))

        types = np.unique(val.get_atomic_numbers(), return_inverse=True)[1]
        cart = val.get_positions(wrap=True)
        force = val.get_array(force_name)

        f.write('AtomData: id type cartes_x cartes_y cartes_z fx fy fz\n')
        for ict, ival in enumerate(val):
            f.write(('          {:<5d} {:<5d} ' + '{:<10.8f}   '*6 + '\n').format(
                    ict+1, types[ict], *cart[ict], *force[ict]))

        f.write('Energy\n{}\n'.format(val.info[energy_name]))

        if virial_name in val.info.keys():
            if isinstance(val.info[virial_name], np.ndarray):
                stress = np.reshape(val.info[virial_name], (3,3))

                stress = np.concatenate((stress.diagonal(),
                                     stress[1,2], stress[0,2], stress[0,1]),
                                     axis=None)

            f.write('PlusStress: xx     yy     zz     yz     xz     xy\n')
            f.write(('          '+'{}'*6).format(*stress))

        for i,j in val.info.items():
            if i != energy_name and i != virial_name:
                f.write('Feature    {}    {}\n'.format(str(i).replace('\n', ''),
                                                       str(j).replace('\n', '')))
        f.write('END_CFG\n\n')

In [115]:
write_cfg_db('test.cfg', db)

In [122]:
write_cfg_db('/home/joe/Documents/Ge_od/Potentials/Si_myDB_216/train_64_125_216_full.cfg', my, force_name='dft_forces')

In [126]:
write_cfg_db('/home/joe/Documents/Ge_od/Potentials/hp_liquid/hpliq_amoliq/hpliq_amoliq_train.cfg', Ge, force_name='dft_forces')

To do:
* write other util functions for reading cfgs (see ase extxyz for inspiration)
* could then think about contributing this to ase

